In [111]:
import numpy as np
from tkinter import *
from tkinter import ttk
import tkinter.scrolledtext as tkst
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.animation import FuncAnimation
import math
import pandas as pd

marathon_2015_2017 = pd.read_csv('./marathon_merge[2015~2017].csv')

In [112]:
record = pd.DataFrame(marathon_2015_2017, columns = ['M/F', 'Age', 'Pace', '10K', '20K', '30K', 'Official Time']).sort_values(by=['Official Time'])

In [113]:
record['M/F'] = record['M/F'].map({'M': 1, 'F': 0})

In [114]:
record_list = record.values.tolist()

gender_list = ['Female', 'Male']

In [115]:
grad_fig = Figure(figsize=(10, 4), dpi=100)
grad_ax = grad_fig.add_subplot(111)
grad_ax.set_xlim(15, 88)
grad_ax.set_ylim(0, 1300)
grad_ax.set_ylabel('Pace : Runner\'s overall minute per mile pace')
grad_ax.set_xlabel('Age : Age on race day')

g_xdata, g_ydata = [], []
gn, = grad_ax.plot([], [], 'ro')

In [116]:
def histogram() :
    # t_gCbbox.get()의 값은 선택된 'Male', 'Female'
    gender = t_gCbbox.get()
    t_g = int(gender_list.index(gender))
    
    t_a = int(t_aSpbox.get())
    t_p = int(t_pSpbox.get())
    
    if (t_g) :
        gender_color = 'b'
    else :
        gender_color = 'r'
        
    gender_record = record[record['M/F'] == t_g]
    gender_age_record = gender_record[gender_record.Age == t_a]
    gender_age_record_list = gender_age_record.values.tolist()
    
    grad_ax.plot(gender_record.Age, gender_record.Pace, '.', color=gender_color, alpha=0.3)
    grad_ax.plot(t_a, t_p, 'yd')
    stat = gender_age_record['Pace'].describe()
    
    title = 'Gender : ' + gender_list[t_g] + ', Age : ' + str(t_a)
    grad_ax.set_title(title)
    
    grad_ax.annotate('%10s %7i' % ('Count : ', stat[0]), (75, 1200), fontsize=10)
    grad_ax.annotate('%10s %7.3f' % ('Mean : ', stat[1]), (75, 1150), fontsize=10)
    grad_ax.annotate('%10s %7.3f' % ('25% : ', stat[3]), (75, 1100), fontsize=10)
    grad_ax.annotate('%10s %7.3f' % ('75% : ', stat[5]), (75, 1050), fontsize=10)
    
    grad_fig.canvas.draw()

In [117]:
def seconds_to_hhmmss(seconds) :
    hours = seconds // (60*60)
    seconds %= (60*60)
    minutes = seconds // 60
    seconds %= 60
    return '%02i:%02i:%02i' % (hours, minutes, seconds)

In [118]:
def learning() :
    import tensorflow as tf
    import numpy as np
    
    gender = t_gCbbox.get()
    t_g = int(gender_list.index(gender))
    t_a = int(t_aSpbox.get())
    t_p = int(t_pSpbox.get())
    
    t_t = int(t_tSpbox.get())
    t_r = float(t_rSpbox.get())
    
    x_train = [r[0:3] for r in record_list]
    y_train = [ [r[-1]] for r in record_list]
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(1, input_shape=(3,)))
    sgd = tf.keras.optimizers.SGD(lr=t_r)
    model.compile(loss='mse', optimizer=sgd)
    model.summary()
    history = model.fit(np.array(x_train), np.array(y_train), epochs=t_t)
    
    log_ScrolledText.insert(END, '\nGender: ' + gender_list[t_g] + ', Age : ' + str(t_a) + ', Pace: ' + str(t_p) + '\n', 'TITLE')
    log_ScrolledText.insert(END, '\n\nCost Decent\n\n', 'HEADER')
    log_ScrolledText.insert(END, '%20s %20s' % ('Step', 'Cost') + '\n\n')
    
    for step in range(t_t):
        if step % 100 == 0:
            cost_val = history.history['loss'][step]
            log_ScrolledText.insert(END, '%20i %20.5f' % (step, cost_val) + '\n')
            
    winner = [t_g, t_a, t_p]
    time = model.predict(np.array([winner]))
    ml_time = seconds_to_hhmmss(time[0][0]) + '(' + str(time[0][0]) + ')'
    
    log_ScrolledText.insert(END, '%20s' % ('\n\nThe Prediction Records\n\n'), 'HEADER')
    log_ScrolledText.insert(END, '%10s %10s %10s %50s' % ('Gender', 'Age', 'Page', 'Record Prediction(Second) at 42.195km') + '\n\n')
    log_ScrolledText.insert(END, '%10s %10s %10s %50s' % (gender_list[t_g], str(t_a), str(t_p), ml_time) + '\n')   

In [119]:
main = Tk()
main.title('Multi Variable Matrix Linear Regression')
main.geometry()

label = Label(main, text='Multi Variable Matrix Linear Regression')
label.config(font=('굴림', 18))
label.grid(row=0, column=0, columnspan=6)

t_gVal = StringVar(value=gender_list[0])
t_gCbbox = ttk.Combobox(main, textvariable=t_gVal)
t_gCbbox['values'] = gender_list
t_gCbbox.config(state='readonly')
t_gCbbox.grid(row=1, column=1)

t_gLabel = Label(main, text='Gender : ')
t_gLabel.grid(row=1, column=0)

t_aVal = IntVar(value=500)
t_aSpbox = Spinbox(main, textvariable=t_pVal, from_=0, to=1500, increment=1, justify=RIGHT)
t_aSpbox.grid(row=1, column=5)

t_pVal = IntVar(value=500)
t_pSpbox = Spinbox(main, textvariable=t_pVal, from_=0, to=1500, increment=1, justify=RIGHT)
t_pSpbox.grid(row=1, column=5)
t_pLabel = Label(main, text='Pace : ')
t_pLabel.grid(row=1, column=4)
t_tVal = IntVar(value=2000)
t_tSpbox=Spinbox(main, textvariable=t_tVal, from_=0, to=100000, increment=1000, justify=RIGHT)
t_tSpbox.grid(row=2, column=1)
t_tLabel = Label(main, text='Number of train : ')
t_tLabel.grid(row=2, column=0)

t_rVal = DoubleVar(value=1e-6)
t_rSpbox=Spinbox(main,textvariable=t_rVal,from_=0,to=1,increment=1e-6,justify=RIGHT)
t_rSpbox.grid(row=2,column=3)
t_rLabel=Label(main, text='Learning rate : ')
t_rLabel.grid(row=2,column=2)

btn1 = Button(main, text='Histogram', height=2, command=lambda:histogram())
btn1.grid(row=2, column=4, columnspan=1, sticky=(W,E))
btn2 = Button(main, text='Prediction', height=2, command=lambda:learning())
btn2.grid(row=2, column=5, columnspan=1, sticky=(W,E))

grad_canvas = FigureCanvasTkAgg(grad_fig, main)
grad_canvas.get_tk_widget().grid(row=3, column=0, columnspan=6)

log_ScrolledText = tkst.ScrolledText(main, height=15)
log_ScrolledText.grid(row=4, column=0, columnspan=6,  sticky=(N, S, W, E))
log_ScrolledText.configure(font='굴림')
log_ScrolledText.tag_config('RESULT', foreground='blue', font=('굴림', 12))
log_ScrolledText.tag_config('HEADER', foreground='black', font=('굴림', 14))
log_ScrolledText.tag_config('TITLE', foreground='red', font=('굴림', 18))

In [120]:
main.mainloop()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2000
2489/2489 [==============================] - 2s 493us/step - loss: 803266.6706
Epoch 2/2000
2489/2489 [==============================] - 1s 496us/step - loss: 9214.5765
Epoch 3/2000
2489/2489 [==============================] - 1s 492us/step - loss: 6964.5984
Epoch 4/2000
2489/2489 [==============================] - 1s 493us/step - loss: 1907.9445
Epoch 5/2000
2489/2489 [==============================] - 1s 494us/step - loss: 4998.1311
Epoch 6/2000
2489/2489 [==============================] - 1s 496us/step - loss: 842.8498
Epoch 7/2000
2489/2489 [==============================] - 1s 497us/step - loss: 715.5304
Epoch 8/2000
2489/2489 [==

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\JAVA01\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-119-c8aea686ba39>", line 41, in <lambda>
    btn2 = Button(main, text='Prediction', height=2, command=lambda:learning())
  File "<ipython-input-118-a6ca1776d71c>", line 21, in learning
    history = model.fit(np.array(x_train), np.array(y_train), epochs=t_t)
  File "C:\Users\JAVA01\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1100, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\JAVA01\anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "C:\Users\JAVA01\anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 855, in _call
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "C:\Users\JAVA01\anaconda3\lib